In [ ]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel, AsyncOpenAI
from agents.mcp import MCPServerStdio, MCPServerStreamableHttp
import gradio as gr

In [ ]:

mcp_params = []
mcp_http_params = []
mcp_servers = []

In [ ]:
# mcp_params.append({"command": "uvx", "args": ["mcp-server-fetch"]})
mcp_params.append({"command": "uv", "args": ["run", "examples/py/date_mcp_server.py"]})
mcp_params.append({"command": "uv", "args": ["run", "examples/py/pvc_company.py"]}) 

# mcp_http_params.append({"url": "https://mcp.deepwiki.com/mcp"})
mcp_http_params.append({"url": "http://localhost:3011/mcp"})


mcp_servers.extend(MCPServerStdio(params=fetch_params, client_session_timeout_seconds=120) for fetch_params in mcp_params)
mcp_servers.extend(MCPServerStreamableHttp(params=fetch_params, client_session_timeout_seconds=120) for fetch_params in mcp_http_params)

In [ ]:
for server in mcp_servers:
        print(f"Connecting to MCP server: {server}")
        await server.connect()
        tools = await server.list_tools()
        for tool in tools:
            print(f"Tool available on {server}: {tool}")

In [ ]:
modal_name = "gemma-3-12b"
# modal_name = "llama3.2"
# modal_name = "gpt-4o-mini"
# modal_name = "deepseek-r1-distill-qwen-7b"
url = "http://localhost:1234/v1"
# url = "http://localhost:11434/v1"
api_key = "lm-studio"
# api_key = "ollama"

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# Configure the model
client = AsyncOpenAI(base_url=url, api_key=api_key)
# client = AsyncOpenAI(api_key=api_key)

model = OpenAIChatCompletionsModel(
    model=modal_name,
    openai_client=client
)

async def get_researcher(mcp_servers) -> Agent:
# Create the agent
    agent = Agent(
        mcp_servers=mcp_servers,
        name="Assistant",
        instructions="You are a helpful assistant with shortly answering questions.",
        model=model
    )
    return agent

In [ ]:
async def run_chat(message, history):
    print(f"Running chat with message: {message} and history: {history}")
    for server in mcp_servers:
        await server.connect()
    print(f"Connected to MCP servers: {mcp_servers}")

    history.append({"role": "user", "content": message})

    yield history, ""

    researcher = await get_researcher(mcp_servers)
    
    with trace("account_mcp_client"):
        result = await Runner.run(
            researcher,
            history, 
            max_turns=30,
        )

    history.append({"role": "assistant", "content": result.final_output})


    yield history, ""

In [ ]:

with gr.Blocks() as ui:
    chat = gr.Chatbot(type="messages", label="agent chat", height=500)
    chat_history = gr.State([])
    
    txt = gr.Textbox(placeholder="Chat with our AI Assistant:", show_label=False)
    btn = gr.Button("Run")
    
    with gr.Row():
        clear = gr.Button("Clear")

    btn.click(
        fn=run_chat,
        inputs=[txt, chat_history],
        outputs=[chat, txt],
    )
    txt.submit(
        fn=run_chat,
        inputs=[txt, chat_history],
        outputs=[chat, txt],
    )
    clear.click(lambda: ([], []), inputs=None, outputs=[chat, chat_history], queue=False)

ui.launch(inbrowser=True)


### Look at the trace

* https://platform.openai.com/traces
* https://www.stainless.com/mcp/how-to-test-mcp-servers

#### how to test mcp 
> npx @modelcontextprotocol/inspector

* Command: `uv`
* Arguments: `run examples/py/date_mcp_server.py`

#### Questions

1. what is the website of Vorkov PVC company? - test MCP
2. what is my time? - test: AI must ask user name.